In [2]:
#imports

import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(42) #replicability

In [3]:
#getting the data
!python -m wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt


Saved under input (1).txt


In [4]:
#inspecting the input text
with open("input.txt", "r") as f:
    text = f.read()

In [5]:
print("Length of tiny shakespeare (character length) = ", len(text))

Length of tiny shakespeare (character length) =  1115394


In [6]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [7]:
#let's check the unique characters in the text file - this is the vocabulary we are dealing with
vocab = sorted(list(set(text)))
vocab_size = len(vocab)

print((" ".join(vocab)))

print("vocab size = ", vocab_size)


   ! $ & ' , - . 3 : ; ? A B C D E F G H I J K L M N O P Q R S T U V W X Y Z a b c d e f g h i j k l m n o p q r s t u v w x y z
vocab size =  65


### Encoder and Decoder

In [8]:
encoder_look_up = {char : i for i, char in enumerate(vocab)}
decoder_look_up = {i : char for i, char in enumerate(vocab)}

def encode(string):
        return [encoder_look_up[char] for char in string]

def decode(coded_list):
        return "".join(decoder_look_up[code] for code in coded_list)

In [9]:
encode("Hello, how are you?")

[20, 43, 50, 50, 53, 6, 1, 46, 53, 61, 1, 39, 56, 43, 1, 63, 53, 59, 12]

In [10]:
decode(encode("Hello, how are you?"))

'Hello, how are you?'

We can use tiktoken or sentencepiece but we are indeed building from scratch

In [11]:
# Encoding the input and store it in a torch tensor
data = torch.tensor(encode(text), dtype=torch.long) #64-bit integer (signed) - We can also use the unsigned 64 bit integer but the torch documentation says limited support for it, so I am going to use the signed one for sanity
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


### Train Val Split

In [12]:
n = int(0.9 * len(data))
train_data = data[:n] #no shuffling as the sequence is important
val_data = data[n:]

### Chunking

In [13]:
chunk_size = 10

In [14]:
x = train_data[:chunk_size]
y = train_data[1: chunk_size+1]
for i in range(chunk_size):
    context = x[:i+1]
    target = y[i]
    print(" Input = ", context, "target is = ", target)

 Input =  tensor([18]) target is =  tensor(47)
 Input =  tensor([18, 47]) target is =  tensor(56)
 Input =  tensor([18, 47, 56]) target is =  tensor(57)
 Input =  tensor([18, 47, 56, 57]) target is =  tensor(58)
 Input =  tensor([18, 47, 56, 57, 58]) target is =  tensor(1)
 Input =  tensor([18, 47, 56, 57, 58,  1]) target is =  tensor(15)
 Input =  tensor([18, 47, 56, 57, 58,  1, 15]) target is =  tensor(47)
 Input =  tensor([18, 47, 56, 57, 58,  1, 15, 47]) target is =  tensor(58)
 Input =  tensor([18, 47, 56, 57, 58,  1, 15, 47, 58]) target is =  tensor(47)
 Input =  tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47]) target is =  tensor(64)


### Batching

In [15]:
torch.manual_seed(42) #for replicating same results
batch_size = 8 #Parallel Processing in GPU
chunk_size = 10 #Context length

def get_batches(split):
    """
    Get batch of data to either "train" or "val"
    """
    
    data = train_data if split.lower()=="train" else val_data
    positions = torch.randint(len(data)-chunk_size, (batch_size,))
    x = torch.stack([data[i:i+chunk_size] for i in positions])
    y = torch.stack([data[i+1:i+chunk_size+1] for i in positions])
    return x, y

In [16]:
xb, yb = get_batches('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([8, 10])
tensor([[39, 41, 43,  1, 42, 43, 39, 58, 46,  6],
        [57,  1, 42, 53, 52, 43,  6,  0, 13, 52],
        [ 0, 27, 36, 18, 27, 30, 16, 10,  0, 15],
        [43,  0, 48, 53, 63,  1, 41, 56, 53, 61],
        [58, 53,  1, 42, 53,  1, 46, 43, 56, 43],
        [ 0,  0, 28, 56, 53, 60, 53, 57, 58, 10],
        [45, 39, 60, 43,  1, 51, 43,  1, 50, 47],
        [59, 54, 54, 53, 56, 58,  1, 23, 47, 52]])
targets:
torch.Size([8, 10])
tensor([[41, 43,  1, 42, 43, 39, 58, 46,  6,  1],
        [ 1, 42, 53, 52, 43,  6,  0, 13, 52, 42],
        [27, 36, 18, 27, 30, 16, 10,  0, 15, 39],
        [ 0, 48, 53, 63,  1, 41, 56, 53, 61, 52],
        [53,  1, 42, 53,  1, 46, 43, 56, 43,  6],
        [ 0, 28, 56, 53, 60, 53, 57, 58, 10,  0],
        [39, 60, 43,  1, 51, 43,  1, 50, 47, 44],
        [54, 54, 53, 56, 58,  1, 23, 47, 52, 45]])


In [17]:
class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()

        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        #idx and targets are (B, T) tenor of integers
        #logits are scores for the next character in the sequence
        logits = self.token_embedding_table(idx) #(B, T , C) Batch = 8, Time = 10, Channels = Vocab size

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view( B * T, C) # This is done because pytorch expects the channels to be the second dimention
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        #idx is the (B, T) array of index in the current context

        for _ in range(max_new_tokens):
            logits, loss = self(idx) #predictions
            logits = logits[:,-1, :] #Want the lass occurance to pluck out T, now becomes (B, C)
            probs = F.softmax(logits, dim=-1) #(B, C)

            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

        return idx

            
    
m = BigramLanguageModel(vocab_size)
out, loss = m(xb, yb)
print(out.shape)
print(loss)


torch.Size([80, 65])
tensor(4.4397, grad_fn=<NllLossBackward0>)


In [18]:
#Starting the sequence with a zero
idx = torch.zeros((1,1), dtype=torch.long)

#generating the sequence
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


?YcnPI
DbuY-,HJfYNmUVupWmNw;KWF-!U$h3;FsMTyaERu
B3O grba.a;O!ilzrqs:JDq'FRuIMQ'AyNbTmU$bXs3HvKVaI gX


## Training the model

In [19]:
#creating the optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

In [20]:
batch_size = 32
for step in range(10000):
    xb, yb = get_batches("train")

    #evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.472346067428589


In [21]:
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


HAUFit; l ICHed out w war ha yevelise 'tWhowKERARYor add hiull DULY Wod ongozDYRWARHicirds wavente, 


# Mathematical trick in self-attention

In [22]:
B, T, C = 4, 8 , 2 # Batch, Time, Channel
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [23]:
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] #(t,c)
        xbow[b, t] = torch.mean(xprev, 0)

In [24]:
x [0] , xbow[0]

(tensor([[-0.0548,  0.2046],
         [ 0.2189,  1.9393],
         [-1.6876, -1.5939],
         [ 1.3013,  0.0608],
         [ 0.2614, -0.0490],
         [-1.4564, -1.1309],
         [-0.2212,  0.1975],
         [-0.8564,  0.3619]]),
 tensor([[-0.0548,  0.2046],
         [ 0.0820,  1.0720],
         [-0.5078,  0.1833],
         [-0.0556,  0.1527],
         [ 0.0078,  0.1124],
         [-0.2362, -0.0949],
         [-0.2341, -0.0531],
         [-0.3119, -0.0012]]))

In [29]:
torch.mean(x[0], 0) == xbow[0][-1]

tensor([True, True])

In [31]:
torch.tril(torch.ones(3, 3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [30]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [ ]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (T, T) @ (B, T, C) = (B, T , T) @ (B, T, C) ---> (B, T, C) pytorch adds the B element

In [34]:
torch.allclose(xbow, xbow2)

True

In [36]:
#using softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril==0, float("-inf"))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow3, xbow)

True

## Self Attention

In [37]:
B , T , C = 4, 8 , 32 #batch, time, channels
x = torch.randn(B, T, C)

tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril==0, float("-inf"))
wei = F.softmax(wei, dim=-1)
out = wei @ x

out.shape


torch.Size([4, 8, 32])

In [38]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [39]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

### Single head of attention mechanism

In [ ]:
B , T , C = 4, 8 , 32 #batch, time, channels
x = torch.randn(B, T, C)

#single head performing self attention
head_size = 16
key = nn.Linear(C, head_size, bias= False)
query = nn.Linear(C, head_size, bias= False)
value = nn.Linear(C, head_size, bias= False)

k = key(x) # (B, T , 16)
q = query(x) # (B, T , 16)
v = value(x) # (B, T , 16)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) --> (B, T, T)

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril==0, float("-inf"))
wei = F.softmax(wei, dim=-1)
out = wei @ v

out.shape


torch.Size([4, 8, 32])

In [42]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.9787, 0.0213, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6249, 0.2793, 0.0958, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3000, 0.0589, 0.0931, 0.5480, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0569, 0.1342, 0.0160, 0.7529, 0.0400, 0.0000, 0.0000, 0.0000],
        [0.5191, 0.1521, 0.1175, 0.0376, 0.0641, 0.1096, 0.0000, 0.0000],
        [0.0206, 0.1584, 0.2149, 0.1271, 0.0046, 0.0340, 0.4404, 0.0000],
        [0.0279, 0.2048, 0.0871, 0.0243, 0.0238, 0.5413, 0.0403, 0.0504]],
       grad_fn=<SelectBackward0>)

**Next we improve this code in the GPT.py file**